In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", mlm=False)
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [ ]:
import datasets

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(examples):
    return tokenizer(examples['История болезни'], truncation=True, max_length=512, padding="max_length")

csv_file = 'data/ru/all_data.csv'
studmed_ds_train = datasets.load_dataset('csv', data_files=csv_file, split='train[:80%]').shuffle()
studmed_ds_test = datasets.load_dataset('csv', data_files=csv_file, split='train[80%:]').shuffle()

tokenized_datasets_train = studmed_ds_train.map(tokenize_function, batched=True)
tokenized_datasets_test = studmed_ds_test.map(tokenize_function, batched=True)

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('model')



In [3]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", mlm=False)
model = AutoModelForCausalLM.from_pretrained("model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import pipeline

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [15]:
s = "Жалобы на боль в нижних отделах живота, больше справа, тошнота, однократная рвота Заболел 1 сутки назад, когда отметил боль в животе, позже боль сместилась в нижние отделы, присоединилась тошнота, рвота. Самостоятельно принимала кеторол с кратковременным положительным эффектом. В связи с ухудшением состояния бригадой"
generator(s, max_length=512, num_beams=5, no_repeat_ngram_size=2, 
    num_return_sequences=1, early_stopping=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Жалобы на боль в нижних отделах живота, больше справа, тошнота, однократная рвота Заболел 1 сутки назад, когда отметил боль в животе, позже боль сместилась в нижние отделы, присоединилась тошнота, рвота. Самостоятельно принимала кеторол с кратковременным положительным эффектом. В связи с ухудшением состояния бригадой был направлен на консультацию к гастроэнтерологу. Диагноз был поставлен на основании данных лабораторных и инструментальных исследований. Общий анализ крови от <DATE>г. Заключительный диагноз Заключение Дифференциальный диагноз 1. Жалобы больного 2. Объективное исследование 3. Лабораторные исследования 4. Клинический диагноз 5. Лечение 6. Дневник 7. Эпикриз 8. Список использованной литературы 9. Анамнез заболевания 10. История жизни 11. Данные объективного исследования 12. Литература 13. Краткие биографические данные 14. Дополнительные методы исследования 15. Результаты дополнительных исследований 16. Дыхательная система 17. Сердечнососудистая система 

In [18]:
model.config

GPT2Config {
  "_name_or_path": "model",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "use_cache": true,
  "vocab_size": 50264
}

In [27]:
model.push_to_hub('anechaev/ru_med_gpt3sm_based_on_gpt2')

/Users/andrei/miniforge3/envs/nlp/lib/python3.9/site-packages/huggingface_hub/hf_api.py:1001: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/anechaev/ru_med_gpt3sm_based_on_gpt2 into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Download file pytorch_model.bin: 100%|██████████| 526M/526M [10:46<00:00, 852kB/s]  
Clean file pytorch_model.bin: 100%|██████████| 526M/526M [05:26<00:00, 1.69MB/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 